<a href="https://colab.research.google.com/github/PedroPertusi/afd-monitor-jogadores/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# !python -m pip install selenium
# !python -m pip install webdriver_manager
# !python -m pip install requests
!python -m pip install BeautifulSoup4

In [2]:
!sudo apt update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo dpkg -i google-chrome-stable_current_amd64.deb
!sudo apt-get -f install
!google-chrome-stable --version

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,665 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,926 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,211 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,546 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,720 kB]
Get:13 https://ppa.launchpadconten

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.set_page_load_timeout(60)

In [9]:
def baixar_pagina(url):
  response = requests.get(url)
  response.encoding = "utf-8"
  html = response.text
  return html

In [49]:
def extrair_links_noticias_post(url):
  html = baixar_pagina(url)
  soup = BeautifulSoup(html, 'html.parser')
  title = soup.find_all('h2')
  links_noticias = []
  for t in title:
    if t.find('a').get('href'):
      links_noticias.append(t.find('a').get('href'))
  return links_noticias

In [95]:
def extrair_conteudo_noticia(url):
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    ps = soup.find_all('p', class_='content-text__container')

    textos = []
    for p in ps:
        # descarta parágrafos que contenham intertítulo ou lista
        if p.find('div', class_='content-intertitle') or p.find('ul'):
            continue
        # extrai texto limpo (inclui <a>, <strong>, etc)
        textos.append(p.get_text(" ", strip=True))
    return textos

In [105]:
galo = "https://ge.globo.com/futebol/times/atletico-mg/"
noticias = extrair_links_noticias_post(galo)

conteudos = []
for n in noticias:
  conteudos.append(extrair_conteudo_noticia(n))


c = [" ".join(c) for c in conteudos]

'Após atrasar o pagamento pela compra de Deyverson, o Atlético-MG ainda não efetuou os vencimentos da compra do atacante Cuello, ex-Athletico-PR. A informação foi revelada pelo presidente do clube paranaense, Mario Celso Petraglia. Em áudio vazado na quinta-feira, o dirigente revela que o Alvinegro não pagou nenhuma prestação do acordo pelo atacante. O Atlético fechou a compra do argentino por cerca de seis milhões de dólares (R$ 36 milhões na época). O ge procurou o clube mineiro, que disse que não comenta assuntos internos. Cuello foi o quinto reforço do Atlético para a temporada. Chegou para preencher uma carência no elenco: extremo de velocidade. Em pouco tempo, o argentino conquistou a titularidade e tem sido um dos destaques do time. Em 21 jogos, marcou três gols e anotou quatro assistências.'

In [5]:
url = 'https://ge.globo.com/'
driver.get(url)
driver.implicitly_wait(10)

div_times = driver.find_elements


# <div class="mosaico__equipes" id="tab-content-1"> <div class="mosaico__equipes-items">